In [59]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from math import pi

iris = pd.read_csv('Iris.csv')
iris.drop('Id',axis=1,inplace=True)

In [60]:
def MLestimator(dataset):
    mean = np.mean(dataset,axis=0)
    variance = np.var(dataset,axis=0)
    return(mean,variance)
def variance_squaremat(var):
    return(np.multiply(var,[[1,0],[0,1]]))
def BayesClassifier(test_point,mean,variance):
    mean = np.array([mean])
    X = test_point-mean
    p1 = ((1/(2*pi)**0.5)/(np.linalg.det(variance)))**0.5
    p2 = np.exp(-0.5*np.matmul(np.matmul(X,np.linalg.inv(variance)),np.transpose(X)))
    P = (1/3)*p1*p2     # 1/3 is the prior probability
    return(P)
def fold_i_of_k(dataset, i, k):
    n = len(dataset)
    return dataset[n*(i-1)//k:n*i//k]

In [61]:
def compute_confusion_matrix(true, pred):
  '''Computes a confusion matrix using numpy for two np.arrays
  true and pred.

  Results are identical (and similar in computation time) to: 
    "from sklearn.metrics import confusion_matrix"

  However, this function avoids the dependency on sklearn.'''

  K = len(np.unique(true)) # Number of classes 
  result = np.zeros((K, K))

  for i in range(len(true)):
    result[true[i]][pred[i]] += 1

  return result

In [62]:
# separating the dataset based on the type of flower
iris_virginica = iris.loc[iris.Species=='Iris-virginica']
iris_setosa = iris.loc[iris.Species =='Iris-setosa']
iris_versicolor = iris.loc[iris.Species=='Iris-versicolor']

    


In [91]:
acc=0
for i in range (1,5):
    test_virginica=fold_i_of_k(iris_virginica,i,4)
    train_virginica=iris_virginica[~iris_virginica.isin(test_virginica)].dropna()
    
    test_setosa=fold_i_of_k(iris_setosa,i,4)
    train_setosa=iris_setosa[~iris_setosa.isin(test_setosa)].dropna()
    
    test_versicolor=fold_i_of_k(iris_versicolor,i,4)
    train_versicolor=iris_versicolor[~iris_versicolor.isin(test_versicolor)].dropna()
    
    train_virginica.drop(['Species','SepalLengthCm','SepalWidthCm'],axis=1,inplace=True)
    train_setosa.drop(['Species','SepalLengthCm','SepalWidthCm'],axis=1,inplace=True)
    train_versicolor.drop(['Species','SepalLengthCm','SepalWidthCm'],axis=1,inplace=True)
    test_virginica.drop(['SepalLengthCm','SepalWidthCm'],axis=1,inplace=True)
    test_setosa.drop(['SepalLengthCm','SepalWidthCm'],axis=1,inplace=True)
    test_versicolor.drop(['SepalLengthCm','SepalWidthCm'],axis=1,inplace=True)
    test_dataset = pd.concat([test_virginica,test_setosa,test_versicolor])
    test_dataset = test_dataset.reset_index(drop=True)
    test_virginica.drop(['Species'],axis=1,inplace=True)
    test_setosa.drop(['Species'],axis=1,inplace=True)
    test_versicolor.drop(['Species'],axis=1,inplace=True)
    train_virginica = train_virginica.values
    train_setosa = train_setosa.values
    train_versicolor = train_versicolor.values
    test_virginica = test_virginica.values
    test_setosa = test_setosa.values
    test_versicolor = test_versicolor.values
    mean_virginica,variance_virginica= MLestimator(train_virginica)
    mean_setosa,variance_setosa= MLestimator(train_setosa)
    mean_versicolor,variance_versicolor= MLestimator(train_versicolor)
    
    # converting to covariance form (square matrix)

    variance_virginica = variance_squaremat(variance_virginica)
    variance_setosa = variance_squaremat(variance_setosa)
    variance_versicolor = variance_squaremat(variance_versicolor)
    y_test=[]


    count = 0
    
    pred=[]
    test=[]
    n = 45  # no. of iterations
    for _ in range(n):
        sample = test_dataset.sample()
        s = sample.drop('Species',axis=1,inplace=False)
        s = s.values


        # virginica flower type probability calculation call
        prob_virginica = BayesClassifier(s,mean_virginica,variance_virginica)

        # setosa flower type probability calculation call
        prob_setosa = BayesClassifier(s,mean_setosa,variance_setosa)

        # versicolor flower type probability calculation call
        prob_versicolor = BayesClassifier(s,mean_versicolor,variance_versicolor)

        if(max(prob_virginica,prob_setosa,prob_versicolor)==prob_virginica):
            c = 'Iris-virginica'
        elif(max(prob_virginica,prob_setosa,prob_versicolor)==prob_setosa):
            c = 'Iris-setosa'
        elif(max(prob_virginica,prob_setosa,prob_versicolor)==prob_versicolor):
            c = 'Iris-versicolor'
        s = str(sample.iloc[0,2])
        pred.append(c)

        test.append(s)

        if(s==c):
            count+=1

    print("The accuracy of the Bayes classifier which uses ML estimation is: %0.2f %%" %(count*100/n))
    acc=acc+(count*100/n)
    for i in range(45):
        if test[i]=='Iris-setosa':
            y_test.append(0)
        if test[i]=='Iris-versicolor':
            y_test.append(1)
        if test[i]=='Iris-virginica':
            y_test.append(2)
    y_pred=[]
    for i in range(45):
        if pred[i]=='Iris-setosa':
            y_pred.append(0)
        if pred[i]=='Iris-versicolor':
            y_pred.append(1)
        if pred[i]=='Iris-virginica':
            y_pred.append(2)
    print(compute_confusion_matrix(y_test, y_pred))
    
    
print("The accuracy mean of 4 fold cross validation is: %0.2f %%" %(acc/4))

The accuracy of the Bayes classifier which uses ML estimation is: 100.00 %
[[20.  0.  0.]
 [ 0. 10.  0.]
 [ 0.  0. 15.]]
The accuracy of the Bayes classifier which uses ML estimation is: 97.78 %
[[18.  0.  0.]
 [ 0. 12.  0.]
 [ 0.  1. 14.]]
The accuracy of the Bayes classifier which uses ML estimation is: 100.00 %
[[13.  0.  0.]
 [ 0. 19.  0.]
 [ 0.  0. 13.]]
The accuracy of the Bayes classifier which uses ML estimation is: 100.00 %
[[15.  0.  0.]
 [ 0. 17.  0.]
 [ 0.  0. 13.]]
The accuracy mean of 4 fold cross validation is: 99.44 %
